In [1]:
import pandas as pd
import numpy as np
import copy
import pydash

In [2]:
df = pd.DataFrame({'key1':['a', 'a', 'b', 'b', 'a'],
     'key2':['one', 'two', 'one', 'two', 'one'],
     'data1':np.random.randn(5),
     'data2':np.random.randn(5)})

In [3]:
YEAR_LIST = ['2011', '2012', '2013', '2014', '2015',
             '2016', '2017', '2018', '2019', '2020',
             '2021', '2022', '2023', '2024', '2025',
             '2026', '2027', '2028', '2029', '2030'
             ]


COL_NAME_DICT = {
    'Company Name': "gold_company_name",
    'Field ID': 'gold_field_code',
    'Field Name': 'gold_field_name',
    'Name in Case': 'gold_name_in_case',
    # 'Year1'
    # 'Year2'
    'Unit': 'gold_unit',
    'Type(E/R/C)': 'gold_type',
    'Table': 'gold_table',
    'Chapter': 'gold_chapter',
    'page no': 'gold_page_id'
}

In [9]:
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

def get_year_list_and_gold_df(file_name):

    year_list = []

    def get_year(col_name):
        if not col_name.strip().startswith("E"):
            return None
        for year in YEAR_LIST:
            if year in col_name:
                return year
        return None

    df = pd.read_csv(file_name)
    df = df.dropna(axis=0, how='all')
    # df = df.dropna(subset=[file_name_col])
    df = df.fillna("")

    col_mapping_dict = {}

    for csv_col_name in df.columns:
        for key, value in COL_NAME_DICT.items():
            if key not in csv_col_name:
                continue
            else:
                col_mapping_dict[csv_col_name] = value
                break
    for csv_col_name in df.columns:
        year = get_year(csv_col_name)
        if not year:
            continue
        year_list.append(year)
        col_mapping_dict[csv_col_name] = year

    df = df.loc[:, list(col_mapping_dict)]
    df = df.rename(columns=col_mapping_dict)
    
    gold_df = trim_all_columns(df)
    gold_df = gold_df[gold_df.gold_field_code.isin(['E_PL_1', 'E_PL_2'])]
    
    return year_list, gold_df

def get_gold_list(year_list, gold_df):
    gold_tmp_list = gold_df.to_dict("record")
    gold_list = []
    for gold_item in gold_tmp_list:
        tmp_item = copy.deepcopy(gold_item)
        for year in year_list:
            tmp_item.pop(year)

        for year in year_list:
            gold_value = gold_item[year]
            nc_item = copy.deepcopy(tmp_item)
            nc_item["gold_value"] = gold_value
            nc_item["gold_year"] = year

            gold_list.append(nc_item)
    return gold_list

def get_gold_dict_for_year_code(gold_list):

    gold_dict = {}

    gold_year_dict = pydash.group_by(gold_list, ["gold_year"])

    for year, one_year_item_list in gold_year_dict.items():
        y_item_list = [item for item in one_year_item_list if item.get("gold_page_id") != "" and item.get("gold_value") != ""]
        # one_year_dict = pydash.group_by(one_year_info, ["gold_field_code"])
        gold_dict[year] = y_item_list
    return gold_dict

In [10]:
path = '/Users/blair/ghome/6E/work_project/mlar/mlar_offline_dashboard/data/golden_csv'
file_name = f'{path}/Singtel AR 2019.pdf - Human.csv'
year_list, gold_df = get_year_list_and_gold_df(file_name)

gold_list = get_gold_list(year_list, gold_df)
gold_dict = get_gold_dict_for_year_code(gold_list)
gold_dict



{'2019': [{'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'Consolidated Income Statement',
   'gold_chapter': '',
   'gold_page_id': '140',
   'gold_value': '17,371.7',
   'gold_year': '2019'},
  {'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'OPERATING REVENUE',
   'gold_chapter': 'Notes to the Financial Statements',
   'gold_page_id': '170',
   'gold_value': '17,371.7',
   'gold_year': '2019'},
  {'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Group operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'OPERATING REVENUE

In [106]:
gold_year_dict

{'2019': [{'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'Consolidated Income Statement',
   'gold_chapter': '',
   'gold_page_id': 140.0,
   'gold_value': '17,371.7',
   'gold_year': '2019'},
  {'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'OPERATING REVENUE',
   'gold_chapter': 'Notes to the Financial Statements',
   'gold_page_id': 170.0,
   'gold_value': '17,371.7',
   'gold_year': '2019'},
  {'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Group operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'OPERATING REVENUE

In [108]:
one_year_info = gold_year_dict['2019']

pydash.group_by(one_year_info, ["gold_field_code"])

{'E_PL_1': [{'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'Consolidated Income Statement',
   'gold_chapter': '',
   'gold_page_id': 140.0,
   'gold_value': '17,371.7',
   'gold_year': '2019'},
  {'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'OPERATING REVENUE',
   'gold_chapter': 'Notes to the Financial Statements',
   'gold_page_id': 170.0,
   'gold_value': '17,371.7',
   'gold_year': '2019'},
  {'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Group operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'OPERATING REVEN

In [50]:
gold_list = gold_df.to_dict("record")

pydash.group_by(gold_list, "gold_field_code")

{'E_PL_1': [{'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'Consolidated Income Statement',
   'gold_chapter': '',
   'gold_page_id': 140.0,
   '2019': '17,371.7',
   '2018': '17,268.0'},
  {'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'OPERATING REVENUE',
   'gold_chapter': 'Notes to the Financial Statements',
   'gold_page_id': 170.0,
   '2019': '17,371.7',
   '2018': '17,268.0'},
  {'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Group operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'OPERATING REVENUE',
   'gold_

In [20]:
g_dict = (dict(list(grouped)))

In [1]:
{'2019': [{'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'Consolidated Income Statement',
   'gold_chapter': '',
   'gold_page_id': '140',
   'gold_value': '17,371.7',
   'gold_year': '2019'},
  {'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'OPERATING REVENUE',
   'gold_chapter': 'Notes to the Financial Statements',
   'gold_page_id': '170',
   'gold_value': '17,371.7',
   'gold_year': '2019'},
  {'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Group operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'OPERATING REVENUE',
   'gold_chapter': 'Group Five-year Financial Summary',
   'gold_page_id': '110',
   'gold_value': '17,372',
   'gold_year': '2019'},
  {'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'SEGMENT INFORMATION',
   'gold_chapter': 'Notes to the Financial Statements',
   'gold_page_id': '231',
   'gold_value': '17,371.7',
   'gold_year': '2019'}]

SyntaxError: unexpected EOF while parsing (<ipython-input-1-db227c9854d7>, line 44)

In [23]:
df = pd.DataFrame({'a':[1,2,3],'b':[1,2,3]})

In [24]:
df

,a,b
0,1,1
1,2,2
2,3,3


In [28]:
df = df.rename(columns={'a':'A','b': 'B2'})

In [29]:
df

,A,B2
0,1,1
1,2,2
2,3,3


In [30]:
df.get_values()

0    1
1    2
2    3
Name: A, dtype: int64

In [31]:
a = {'a1':1, 'b2':2}
a

{'a1': 1, 'b2': 2}

In [32]:
list(a)

['a1', 'b2']

In [4]:
k = [{'gold_company_name': 'Singtel AR 2019.pdf',
  'gold_field_code': 'E_PL_1',
  'gold_field_name': 'Sales',
  'gold_name_in_case': 'Operating revenue',
  'gold_unit': 'millions, S$',
  'gold_type': 'E',
  'gold_table': 'Consolidated Income Statement',
  'gold_chapter': '',
  'gold_page_id': 140.0,
  'gold_value': '17,371.7',
  'gold_year': '2019'},
     {'gold_company_name': 'Singtel AR 2019.pdf',
  'gold_field_code': 'E_PL_1',
  'gold_field_name': 'Sales',
  'gold_name_in_case': 'Operating revenue',
  'gold_unit': 'millions, S$',
  'gold_type': 'E',
  'gold_table': 'Consolidated Income Statement',
  'gold_chapter': '',
  'gold_page_id': 140.0,
  'value': '17,268.0',
  'gold_year': '2018'}, 
          {'gold_company_name': 'Singtel AR 2019.pdf',
  'gold_field_code': 'E_PL_3',
  'gold_field_name': 'Sales',
  'gold_name_in_case': 'Operating revenue',
  'gold_unit': 'millions, S$',
  'gold_type': 'E',
  'gold_table': 'Consolidated Income Statement',
  'gold_chapter': '',
  'gold_page_id': 140.0,
  'value': '17,268.0',
  'gold_year': '2018'},
     {'gold_company_name': 'Singtel AR 2019.pdf',
  'gold_field_code': 'E_PL_1',
  'gold_field_name': 'Sales',
  'gold_name_in_case': 'Operating revenue',
  'gold_unit': 'millions, S$',
  'gold_type': 'E',
  'gold_table': 'Consolidated Income Statement',
  'gold_chapter': '',
  'gold_page_id': 140.0,
  'value': '17,268.0',
  'gold_year': '2018'}
    ]

In [5]:
k

[{'gold_company_name': 'Singtel AR 2019.pdf',
  'gold_field_code': 'E_PL_1',
  'gold_field_name': 'Sales',
  'gold_name_in_case': 'Operating revenue',
  'gold_unit': 'millions, S$',
  'gold_type': 'E',
  'gold_table': 'Consolidated Income Statement',
  'gold_chapter': '',
  'gold_page_id': 140.0,
  'gold_value': '17,371.7',
  'gold_year': '2019'},
 {'gold_company_name': 'Singtel AR 2019.pdf',
  'gold_field_code': 'E_PL_1',
  'gold_field_name': 'Sales',
  'gold_name_in_case': 'Operating revenue',
  'gold_unit': 'millions, S$',
  'gold_type': 'E',
  'gold_table': 'Consolidated Income Statement',
  'gold_chapter': '',
  'gold_page_id': 140.0,
  'value': '17,268.0',
  'gold_year': '2018'},
 {'gold_company_name': 'Singtel AR 2019.pdf',
  'gold_field_code': 'E_PL_3',
  'gold_field_name': 'Sales',
  'gold_name_in_case': 'Operating revenue',
  'gold_unit': 'millions, S$',
  'gold_type': 'E',
  'gold_table': 'Consolidated Income Statement',
  'gold_chapter': '',
  'gold_page_id': 140.0,
  'valu

In [7]:
import pydash
d = pydash.group_by(k, ["gold_field_code"])

In [8]:
d

{'E_PL_1': [{'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'Consolidated Income Statement',
   'gold_chapter': '',
   'gold_page_id': 140.0,
   'gold_value': '17,371.7',
   'gold_year': '2019'},
  {'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'Consolidated Income Statement',
   'gold_chapter': '',
   'gold_page_id': 140.0,
   'value': '17,268.0',
   'gold_year': '2018'},
  {'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'Consolidated Income Statement',
   'gold_chapte

In [83]:
pydash.group_by(d['2018'], ["gold_field_code"])

{'E_PL_1': [{'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'Consolidated Income Statement',
   'gold_chapter': '',
   'gold_page_id': 140.0,
   'value': '17,268.0',
   'gold_year': '2018'},
  {'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_1',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'Consolidated Income Statement',
   'gold_chapter': '',
   'gold_page_id': 140.0,
   'value': '17,268.0',
   'gold_year': '2018'}],
 'E_PL_3': [{'gold_company_name': 'Singtel AR 2019.pdf',
   'gold_field_code': 'E_PL_3',
   'gold_field_name': 'Sales',
   'gold_name_in_case': 'Operating revenue',
   'gold_unit': 'millions, S$',
   'gold_type': 'E',
   'gold_table': 'Consolidated Income Statement',
   'gold_

In [86]:
t_list = [
        {
            "field_name": "sales",
            "field_code": "E_PL_1",
            "value": "45,743",
            "page_id": 1,
            "table_id": 1,
            "paragraph_id": None,
            "strategy": "rule"
        },
        {
            "field_name": "cost of sales",
            "field_code": "E_PL_2",
            "value": "(43,477)",
            "page_id": 1,
            "table_id": 1,
            "paragraph_id": None,
            "strategy": "rule"
        },
        {
            "field_name": "gross profit",
            "field_code": "E_PL_5",
            "value": "2,266",
            "page_id": 1,
            "table_id": 2,
            "paragraph_id": None,
            "strategy": "rule"
        },
        {
            "field_name": "interest expense",
            "field_code": "E_PL_21",
            "value": "(339)",
            "page_id": 1,
            "table_id": 1,
            "paragraph_id": None,
            "strategy": "rule"
        },
        {
            "field_name": "taxation",
            "field_code": "E_PL_37",
            "value": "(179)",
            "page_id": 1,
            "table_id": 1,
            "paragraph_id": None,
            "strategy": "rule"
        }
    ]

In [87]:
pydash.group_by(t_list, ["field_code"])

{'E_PL_1': [{'field_name': 'sales',
   'field_code': 'E_PL_1',
   'value': '45,743',
   'page_id': 1,
   'table_id': 1,
   'paragraph_id': None,
   'strategy': 'rule'}],
 'E_PL_2': [{'field_name': 'cost of sales',
   'field_code': 'E_PL_2',
   'value': '(43,477)',
   'page_id': 1,
   'table_id': 1,
   'paragraph_id': None,
   'strategy': 'rule'}],
 'E_PL_5': [{'field_name': 'gross profit',
   'field_code': 'E_PL_5',
   'value': '2,266',
   'page_id': 1,
   'table_id': 2,
   'paragraph_id': None,
   'strategy': 'rule'}],
 'E_PL_21': [{'field_name': 'interest expense',
   'field_code': 'E_PL_21',
   'value': '(339)',
   'page_id': 1,
   'table_id': 1,
   'paragraph_id': None,
   'strategy': 'rule'}],
 'E_PL_37': [{'field_name': 'taxation',
   'field_code': 'E_PL_37',
   'value': '(179)',
   'page_id': 1,
   'table_id': 1,
   'paragraph_id': None,
   'strategy': 'rule'}]}